In [1]:
import oyaml as yaml
import os
import datetime as dt
import numpy as np
import re

In [2]:
suffix='AT0_05' #0_008 #'AT1' #'AT0_1' # 'AT0_01' #'AT0'
ssbase='/data/eolson/results/MEOPAR/SS-run-sets/TRACE3/'
pathbase=r'/data/eolson/results/MEOPAR/SS-run-sets/TRACE3'
base_yaml='TRACE3.yaml'
base_namelist_cfg='namelist_cfg_namrun'
namelist_top_cfg_norst='namelist_top_cfg_trace3_norst'
namelist_top_cfg_rst='namelist_top_cfg_trace3_rst'
namelist_trace3_cfg='namelist_trace3_cfg_'+suffix
filedef1h='file_def.xml_1h_p'#'file_def.xml_1h'
filedef1d='file_def.xml_1d_p'#'file_def.xml_1d'

ts_per_day=2160
rundays=20 # per segment
account='rrg-allen'
config_name='TRACE3'
walltime='6:00:00'
startDate=dt.datetime(2017,1,1)
endDate=dt.datetime(2017,12,1)
restart0Phys='$MEOPAR/NEMO-forcing/restarts/HC201812/01jan17/SalishSea_02473200_restart.nc' ###

runName='TRACE3_'+suffix
resultsLoc='$SCRATCH/results/'+runName

targetDates=(dt.datetime(2017,4,10),dt.datetime(2017,5,31),dt.datetime(2017,11,1))

In [3]:
startTS=int(restart0Phys[-19:-11])
print(startTS)

2473200


### Produce list of start and end dates and timesteps, then loop

In [4]:
datesBegin=list()
tsBegin=list()
tsEnd=list()
datesBegin_i=startDate
tsBegin_i=startTS
while datesBegin_i<endDate:
    datesBegin.append(datesBegin_i)
    tsBegin.append(tsBegin_i)
    tsBegin_i=tsBegin_i+ts_per_day*rundays
    datesBegin_i=datesBegin_i+dt.timedelta(days=rundays)
    tsEnd.append(tsBegin_i)

In [5]:
for ind, dd, ts0, ts1 in zip(np.arange(0,len(tsBegin)),datesBegin,tsBegin,tsEnd):
    ### Start by producing yaml files
    hourly=False
    for itarget in targetDates:
        if dd<=itarget and (dd+dt.timedelta(days=20))>itarget:
            hourly=True
    print(ind, dd,ts0,ts1, hourly)
    with open(os.path.join(pathbase,base_yaml)) as file:
        fyaml = yaml.load(file, Loader=yaml.FullLoader)

    # set same for all:
    fyaml['account']=account
    fyaml['config_name']=config_name
    fyaml['walltime']=walltime
    fyaml['namelists']['namelist_trace3_cfg']=[namelist_trace3_cfg,]

    ## conditional restarts:
    if ind==0:
        fyaml['restart']['restart.nc']=restart0Phys
        del fyaml['restart']['restart_trc.nc']
        fyaml['namelists']['namelist_top_cfg']=[namelist_top_cfg_norst,]
    else:
        fyaml['restart']['restart.nc']=resultsLoc+f'/SalishSea_{ts0:08d}_restart.nc'
        fyaml['restart']['restart_trc.nc']=resultsLoc+f'/SalishSea_{ts0:08d}_restart_trc.nc'
        fyaml['namelists']['namelist_top_cfg']=[namelist_top_cfg_rst,]

    ## conditional namelists:
    dstr=dd.strftime('%Y%m%d')
    namcfg=f'namelist_cfgs/namelist_cfg_namrun_{dstr}_{rundays}d'
    fyaml['namelists']['namelist_cfg'][0]=namcfg
    #  conditional id
    run_id=f'{runName}_{ind}'
    fyaml['run_id']=run_id

    # conditional filedef.xml
    hourly=False
    for itarget in targetDates:
        if dd<=itarget and (dd+dt.timedelta(days=20))>itarget:
            hourly=True
    if hourly:
        fyaml['output']['filedefs']=filedef1h
    else: 
        fyaml['output']['filedefs']=filedef1d

    with open(os.path.join(pathbase,run_id+'.yaml'), 'w') as file:
        out = yaml.dump(fyaml, file,default_flow_style=False)
        
    ### namelists
    if not os.path.exists(os.path.join(ssbase,namcfg)):
        with open(os.path.join(ssbase,base_namelist_cfg), 'r') as file:
            namrun = file.readlines()
        namrun2=namrun.copy()
        
        spaces='                '
        datestr=dd.strftime('%Y%m%d')
        for iline, line in enumerate(namrun):
            if re.match('\s+nn_it000',line):
                ll=len(re.search('=\s+\d+',line).group())-2
                namrun2[iline]=re.sub('(?==\s)=\s+\d+','= '+f'{spaces} {ts0+1}'[-ll:],line)
            elif re.match('\s+nn_itend',line):
                ll=len(re.search('=\s+\d+',line).group())-2
                namrun2[iline]=re.sub('(?==\s)=\s+\d+','= '+f'{spaces} {ts1}'[-ll:],line)
            elif re.match('\s+nn_date0',line):
                ll=len(re.search('=\s+\d+',line).group())-2
                namrun2[iline]=re.sub('(?==\s)=\s+\d+','= '+f'{spaces} {datestr}'[-ll:],line)
        
        with open(os.path.join(ssbase,namcfg), 'w') as fout:
            fout.writelines( namrun2 )

0 2017-01-01 00:00:00 2473200 2516400 False
1 2017-01-21 00:00:00 2516400 2559600 False
2 2017-02-10 00:00:00 2559600 2602800 False
3 2017-03-02 00:00:00 2602800 2646000 False
4 2017-03-22 00:00:00 2646000 2689200 True
5 2017-04-11 00:00:00 2689200 2732400 False
6 2017-05-01 00:00:00 2732400 2775600 False
7 2017-05-21 00:00:00 2775600 2818800 True
8 2017-06-10 00:00:00 2818800 2862000 False
9 2017-06-30 00:00:00 2862000 2905200 False
10 2017-07-20 00:00:00 2905200 2948400 False
11 2017-08-09 00:00:00 2948400 2991600 False
12 2017-08-29 00:00:00 2991600 3034800 False
13 2017-09-18 00:00:00 3034800 3078000 False
14 2017-10-08 00:00:00 3078000 3121200 False
15 2017-10-28 00:00:00 3121200 3164400 True
16 2017-11-17 00:00:00 3164400 3207600 False
